In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as pplt
from matplotlib.font_manager import FontProperties
# Loading the data
xl = pd.ExcelFile("/Users/isabel/Desktop/koito/crime_in_the_united_states.xls")
#print xl.sheet_names

# saving data as dataframe 
df2 = xl.parse(xl.sheet_names[0])

# data cleaning
df = df2.ix[2:]
df.columns = df.iloc[0]
dff = df.reindex(df.index.drop(2))

dfm = dff.dropna(0,'all')
dfm = dfm.apply(pd.to_numeric, errors='ignore')
df1A = dfm.ix[33:35]


In [ ]:
# Cleaning data for table 1A
df1A.ix[33,1] = dfm.ix[22,1]/dfm.ix[21,1] *100 -100   #population ratio 2013/2012
df1A.ix[34,1] = dfm.ix[22,1]/dfm.ix[18,1]  *100 -100 #population ratio 2013/2009
df1A.ix[35,1] = dfm.ix[22,1]/dfm.ix[13,1] *100 -100  #population ratio 2013/2004
df1A = df1A.dropna(1,'all')

df1A=df1A.rename(columns = {'Population1':'Relative population change'})

df1A

In [ ]:
# Data fixing for main dataframe + fixing the cloumn names
dfm = dfm.ix[:22]
dfm = dfm.dropna(1,'all')
dfm = dfm.ix[:, :20]
dfm.ix[10,0]=2001
dfm.ix[21,0]=2012
dfm = dfm.apply(pd.to_numeric, errors='ignore')

for idxs in range(len(dfm.columns)):
    str1 = dfm.columns[idxs]
    str2 = dfm.columns[idxs].replace('\n', ' ')
    dfm=dfm.rename(columns ={str1:str2})
    #print str2

#dfm.columns


In [ ]:
# preparing the split of columns for absolute numbers and relative numbers
nruns = (len(dfm.columns)-2)*0.5

st_idx = 2;
newlist_idx = np.linspace(2,(len(dfm.columns)-2),nruns)

newlist = dfm.columns[newlist_idx.astype(int)]
newlist_idx = np.linspace(3,(len(dfm.columns)-1),nruns)
newlistRate = dfm.columns[newlist_idx.astype(int)]
#print newlistRate

In [ ]:
# Plot all absolute numbers except population vs year

fontP = FontProperties()
fontP.set_size('small')
fig = pplt.figure()
ax = pplt.subplot(111)

for item in newlist:
    ax.plot(dfm['Year'],dfm[item])
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax.legend(title="Total crime occurance",loc='center left', prop = fontP, bbox_to_anchor=(1, 0.5))
pplt.show()

In [ ]:
# Plot all relative numbers vs year
fig = pplt.figure()
ax = pplt.subplot(111)

for item in newlistRate:
    ax.plot(dfm['Year'],dfm[item])
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax.legend(title="Crime rates",loc='center left', prop = fontP, bbox_to_anchor=(1, 0.5))
pplt.show()

In [ ]:
# Create new data frame for computation results
index = {'absolute (rel) change to 1994','yearly min change (rate)','yearly max change (rate)','yearly avg change (rate)'}
columns = dfm.columns
#print columns
dfR = pd.DataFrame(index=index, columns=columns)
dfR = dfR.drop('Year', 1)
dfR = dfR.drop('Population1', 1)
#print dfR

In [ ]:
# Compared with the first year of recordings in 1994, the rates in all crime categories went down

for idx in range(len(newlistRate)):
    item = newlistRate[idx]
    itemlist = dfm[item]
    rellist = itemlist[:]/itemlist[3]
    #print dfm[item]
    yearlylist = []
    for rateidx in range(len(itemlist)-1):
        #print itemlist
        yearlylist.append(itemlist[rateidx+4]/itemlist[rateidx+3])
    yearlylist.append(np.nan)
    dfR[item]['absolute (rel) change to 1994'] = rellist[22]
    dfR[item]['yearly min change (rate)'] = min(yearlylist)
    dfR[item]['yearly max change (rate)'] = max(yearlylist)
    dfR[item]['yearly avg change (rate)'] = np.nanmean(yearlylist)

for idx in range(len(newlist)):
    item = newlist[idx]
    itemlist = dfm[item]
    rellist = itemlist[:]-itemlist[3]
    #print dfm[item]
    yearlylist = []
    for rateidx in range(len(itemlist)-1):
        #print itemlist
        yearlylist.append(itemlist[rateidx+4]-itemlist[rateidx+3])
    yearlylist.append(np.nan)
    dfR[item]['absolute (rel) change to 1994'] = rellist[22]
    dfR[item]['yearly min change (rate)'] = min(yearlylist)
    dfR[item]['yearly max change (rate)'] = max(yearlylist)
    dfR[item]['yearly avg change (rate)'] = np.nanmean(yearlylist)    
    
dfR.head()


In [ ]:
dfR['Violent crime'].head()

In [ ]:
# Plot computation results

x_dataRaw = dfR[newlistRate]#['yearly min change (rate)']
x_data = x_dataRaw.loc['yearly avg change (rate)']
#print x_data

y_data = dfR[newlistRate]#['yearly min change (rate)']
y_data = y_data.loc['absolute (rel) change to 1994']
#print y_data
fig = pplt.figure()
ax = pplt.subplot(111)
#box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

#ax.legend(title="yearly min change vs absolute since 1994")
pplt.xlabel('yearly change (min,avg,max)')
pplt.ylabel('absolute change(1994 to 2012)')
pplt.plot(x_data, y_data, "or")
x_data = x_dataRaw.loc['yearly min change (rate)']
pplt.plot(x_data, y_data, "ob")
x_data = x_dataRaw.loc['yearly max change (rate)']
pplt.plot(x_data, y_data, "og")

pplt.show()
# Only the yearly change behaves nicely linear compared to the absolute change over all years in the data
# min yearly change and max yearly change behave not linear versus overall change
# so there is 'noise' in the data for how the changes happen. The change behaviour is linear overall